# CrossRef Local - Demo for the LLM Era

Key features that matter for AI research assistants:

1. **ABSTRACTS** - Full text for LLM context (not available in many APIs)
2. **IMPACT FACTOR** - Journal quality assessment  
3. **CITATIONS** - Paper importance metrics
4. **SPEED** - 167M records in milliseconds, no rate limits

In [ ]:
import sys
sys.path.insert(0, '../src')

from crossref_local import search, get, count, info

## Database Overview

In [ ]:
db = info()
print(f"Works:     {db['works']:,}")
print(f"FTS:       {db['fts_indexed']:,}")
print(f"Citations: {db['citations']:,}")

## 1. Abstracts - Full Text for LLM Context

Unlike many APIs, CrossRef includes abstracts - essential for LLMs to understand paper content.

In [ ]:
results = search("hippocampal memory consolidation", limit=3)

for work in results.works:
    print(f"📄 {work.title}")
    print(f"   {work.journal} ({work.year})")
    if work.abstract:
        print(f"   📝 {work.abstract[:300]}...")
    print()

## 2. Impact Factor - Assess Journal Quality

In [ ]:
from crossref_local.impact_factor import ImpactFactorCalculator

with ImpactFactorCalculator() as calc:
    for journal in ["Nature", "Science", "Cell", "PLOS ONE"]:
        result = calc.calculate_impact_factor(journal, target_year=2023)
        if result:
            print(f"{journal}: IF = {result['impact_factor']:.2f}")

## 3. Speed - 167M Records, No Rate Limits

In [ ]:
import time

queries = ["machine learning", "CRISPR", "climate change", "neural network"]

for q in queries:
    start = time.perf_counter()
    n = count(q)
    elapsed = (time.perf_counter() - start) * 1000
    print(f"{q:<20} → {n:>10,} matches in {elapsed:.0f}ms")

## 4. Get Work by DOI

In [ ]:
work = get("10.1038/nature12373")

if work:
    print(f"Title:   {work.title}")
    print(f"Authors: {', '.join(work.authors)}")
    print(f"Year:    {work.year}")
    print(f"Journal: {work.journal}")
    print(f"\nCitation: {work.citation()}")

## Use Case: LLM Research Assistant

Build context for an LLM by retrieving relevant papers with abstracts.

In [ ]:
def build_research_context(topic: str, n_papers: int = 5) -> str:
    """Build LLM context from relevant papers."""
    results = search(topic, limit=n_papers)
    
    context = f"## Research Context: {topic}\n\n"
    
    for i, work in enumerate(results.works, 1):
        context += f"### Paper {i}: {work.title}\n"
        context += f"- Authors: {', '.join(work.authors[:3])}"
        if len(work.authors) > 3:
            context += " et al."
        context += f"\n- Journal: {work.journal} ({work.year})\n"
        if work.abstract:
            context += f"- Abstract: {work.abstract}\n"
        context += f"- DOI: {work.doi}\n\n"
    
    return context

# Example
context = build_research_context("transformer attention mechanism", n_papers=3)
print(context[:2000] + "...")

---

## TODO: Graphing Support

Future features:
- Citation network visualization
- Impact factor trends over time
- Author collaboration networks
- Topic clustering